# Prepare Related Data File
* **About 3 mins may be elapsed**
* Please refer to the 0.LookAt_RawData.ipynb if you see a process of handling a raw data file handled

In [25]:
import boto3
import os
import pandas as pd
import numpy as np

## Create Related Time Series File and Validation File
**IMPORTANT: Shift three days in the future because a weekly interval starts from Monday in Amazon Forecast.This dataset starts from Friday for an weekly interval**

In [26]:
data_dir = 'data'
train_file_name = 'train.csv'
features_file_name = 'features.csv'
store_file_name = 'stores.csv'
feature_data = pd.read_csv(os.path.join(data_dir,features_file_name))
feature_df = feature_data.copy()
# Shift three days in the future 
feature_df.Date = pd.to_datetime(feature_df['Date']) + pd.DateOffset(days=3)



In [27]:
feature_df = feature_df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'], axis=1)
# Drop CPI, Unemployment because of missing values
# Drop isHoliday because isHoliday exists in the built-in data of AWS Forecast
feature_df = feature_df.drop(['CPI','Unemployment','IsHoliday'], axis=1)



In [28]:
feature_df = feature_df.rename(columns={'Store':'item_id','Date':'date'})
feature_df.reset_index(inplace=True)
feature_df = feature_df.drop('index', axis=1)
feature_df = feature_df.set_index('date')
feature_df.item_id = feature_df.item_id.astype(str)
feature_df.index = pd.to_datetime(feature_df.index, format = '%Y-%m-%d')
cols = ['Temperature','Fuel_Price','item_id']
feature_df = feature_df[cols]
feature_df.head()

,Temperature,Fuel_Price,item_id
date,,,
2010-02-08,42.31,2.572,1
2010-02-15,38.51,2.548,1
2010-02-22,39.93,2.514,1
2010-03-01,46.63,2.561,1
2010-03-08,46.50,2.625,1


## Save related data file

In [29]:
end_val_date = '2012-10-29'

In [30]:
related_feature_df = feature_df[feature_df.index < end_val_date]
#validation_stores_sales = stores_sales[stores_sales.index >= '2011-10-01']


In [31]:
# With the data in a great state, save it off as a CSV
related_time_series_filename = "related_time_series.csv"
related_time_series_path = data_dir + "/" + related_time_series_filename
related_feature_df.to_csv(related_time_series_path, header=False)

In [32]:
%store related_time_series_filename
%store related_time_series_path
%store related_feature_df


Stored 'related_time_series_filename' (str)
Stored 'related_time_series_path' (str)
Stored 'related_feature_df' (DataFrame)
